# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('homepage_actions.csv')
df.head(20)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [5]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [14]:
#Your code here
click = set(df[df.action == 'click']['id'].unique())
views = set(df[df.action == 'view']['id'].unique())
nclick = len(click)
nviews = len(views)
print('Views: {} \t Clicks: {}'.format(nviews, nclick))
print('Number of viewers that did not click {}'.format(len(views-click)))
print('Number of clickers that did not view {}'.format(len(click-views)))

Views: 6328 	 Clicks: 1860
Number of viewers that did not click 4468
Number of clickers that did not view 0


In [15]:
expt = set(df[df.action == 'experiment']['id'].unique())
cont = set(df[df.action == 'control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(expt&cont)))

Overlap of experiment and control groups: 0


In [ ]:
'''
No clicks that did not view and no overlap between the control and experimental groups. Good data to start analysing.
'''

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [16]:
df['count'] = 1

In [23]:
#make a binary table for control and experimental groups with the actions (click or view) seperated into rows.
control = df[df.group == 'control'].pivot(index='id', columns = 'action', values = 'count')
control = control.fillna(value=0)

experiment = df[df.group == 'experiment'].pivot(index='id', columns = 'action', values = 'count')
experiment = experiment.fillna(value=0)

print("Sample sizes \t Control: {} \t Experimental: {}".format(len(control), len(experiment)))
print("Total Clicks \t Control: {} \t Experimental: {}".format(control['click'].sum(), experiment['click'].sum()))
print("Click Rate \t Control: {} \t Experimental: {}".format(control['click'].mean(), experiment['click'].mean()))


Sample sizes 	 Control: 3332 	 Experimental: 2996
Total Clicks 	 Control: 932.0 	 Experimental: 928.0
Click Rate 	 Control: 0.2797118847539016 	 Experimental: 0.3097463284379172


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [24]:
freqCont = control.click.mean()
expected_experiment_num_clicks = freqCont * len(experiment)
expected_experiment_num_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [25]:
n = len(experiment)
p = freqCont
var = n*p*(1-p)
std = np.sqrt(var)
std

24.568547907005815

In [28]:
experiment_num_clicks = experiment.click.sum()
zscore = (experiment_num_clicks - expected_experiment_num_clicks) / std
zscore

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [29]:
import scipy.stats as stats

In [30]:
#Your code here
pvalue = 1 - stats.norm.cdf(zscore)
pvalue

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [ ]:
'''
Yes, the p-value is well below the critical value, thus we can reject the null hypothesis and thus support that 
the experimental page has a higher probability of having viewers click. 
'''

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.